In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import accuracy_score

In [2]:
%run alzheimer.ipynb

In [3]:
def prepare_dataset_feat_extr_resize_to_model(x, model_input_shape):
    """ Redefine as dimensões da entrada para ser compatível ao modelo da extração de características. 

    Retorna o tamanho da entrada e a distribuição das classes 

    Args:
        x: entrada
        model_input_shape: dimensões esperadas pelo modelo de extração de características
        
    Returns:
        Entrada redimensionada
    """

    X = []
    for i in range(0, x.shape[0]):
        # A imagem na escala de cinza é replicada para os 3 canais de cor, pois os modelos de extração de características esperam imagens coloridas.
        img = Image.fromarray(x[i], mode="RGB")
        X.append(np.array(img.resize(size=model_input_shape)))
    return np.array(X)


def prepare_dataset_feat_extr_input(x, input_shape, model_input_shape):
    """ Prepara a entrada para a extração de características

    Reordenadas as dimensões da entrada e ajusta o seu tamanho para ser compatível ao modelo da extração de características.

    Args:
        x: entrada
        input_shape: dimensões da entrada
        model_input_shape: dimensões esperadas pelo modelo de extração de características
        
    Returns:
        Entrada preparada para a extração de características
    """

    x_channel_formatted, _ = prepare_dataset_channel_position(
        x.astype('float32'), input_shape)
    return prepare_dataset_feat_extr_resize_to_model(x_channel_formatted, model_input_shape)


def prepare_dataset_feat_extr(x, y, input_shape, model_input_shape):
    """ Prepara o conjunto de dados para a extração de características

    Reordenadas as dimensões da entrada e ajusta o seu tamanho para ser compatível ao modelo da extração de características.

    Args:
        x: entrada
        y: saída
        input_shape: dimensões da entrada
        model_input_shape: dimensões esperadas pelo modelo de extração de características
        
    Returns:
        Entrada preparada para a extração de características,
        Saída preparada para a extração de características
    """

    x_prepared = prepare_dataset_feat_extr_input(
        x, input_shape, model_input_shape)
        
    # A saída é inalterada
    return x_prepared, y

In [4]:
def extract_feature(model, x):
    """ Transforma a entrada em um conjunto de características utilizando o modelo

    Faz a previsão da entrada pelo modelo da extração de características. A saída gerada são as características extraídas.

    Args:
        model: modelo de extração de característica
        x: entrada
      
    Returns:
        Características extraídas do modelo
    """

    prediction = np.array(model.predict(x))
    return np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

In [5]:
def feature_extraction(model_and_parameter):
    """ Realiza a extração de características do modelo

    Faz a leitura do conjunto de dados e extrai suas características.

    Args:
        model_and_parameter: variável que contém tanto o modelo como as dimensões da entrada esperada por ele
      
    Returns:
        Entrada de treinamento extraída,
        Saída de treinamento extraída,
        Entrada de teste extraída,
        Saída de teste extraída    
    """
    
    model, model_input_shape = model_and_parameter()

    input_shape, _ = analyse_dataset(TRAIN_DIR_PATH)
    _, _ = analyse_dataset(TEST_DIR_PATH)
    x_trainval, y_trainval = load_dataset(TRAIN_DIR_PATH)
    x_test, y_test = load_dataset(TEST_DIR_PATH)

    x_train, _, y_train, _ = split_dataset(x_trainval, y_trainval)

    x_train_prepared, y_train_prepared = prepare_dataset_feat_extr(
        x_train, y_train, input_shape, model_input_shape)
    x_test_prepared, y_test_prepared = prepare_dataset_feat_extr(
        x_test, y_test, input_shape, model_input_shape)

    x_train_extracted = extract_feature(model, x_train_prepared)
    x_test_extracted = extract_feature(model, x_test_prepared)

    return x_train_extracted, y_train_prepared, x_test_extracted, y_test_prepared


In [6]:
def extract_and_evaluate(model_and_parameter, classifier):
    """ Faz a extração e classificação do conjunto de dados

    Faz a extração de característica e a usa como entrada do classificador. O resultado produzido pelo classificador é o resultado final.

    Args:
        model_and_parameter: variável que contém tanto o modelo como as dimensões da entrada esperada por ele
        classifier: classificador

    Returns:
        Acurácia 
    """
    
    x_train, y_train, x_test, y_test = feature_extraction(model_and_parameter)
    acc , _ = classifier(x_train, y_train, x_test, y_test)
    return acc,

In [7]:
def save_evaluation_feat_extract(evalution_name, all_scores, table):
    """ Salva o resultado de uma avaliação do classificador
    
    Salva o resultado de uma avaliação do classificador, tanto em um formato JSON como em uma tabela para facilitar a leitura.

    Args:
        model_name: nome do diretório do modelo em que a avaliação será salva
        evalution_name: nome da avaliação
        all_scores: resultado da avaliação
        table: tabela do resultado da avaliação
        
    Returns:
        Não há
    """
    
    result_directory = RESULT_PATH
    if not os.path.exists(result_directory):
        raise ValueError("Folder not found.")
    
    # O resultado é salvo em um arquivo com o nome da avaliação e o sufixo "_score" no formato .json.
    score_path = os.path.join(result_directory, evalution_name + '_score.json')
    with open(score_path, 'w') as f:
        json.dump(all_scores, f, ensure_ascii=False, indent=4)

    # A tabela é salva em um arquivo com o nome da avaliação e o sufixo "_score_summary" no formato .txt.
    pretty_score_path = os.path.join(result_directory, evalution_name + '_score_summary.txt')

    with open(pretty_score_path,'w') as f:
        f.write(table.get_string())

def plot_evaluation_roc_feat_extract(evaluation_name, classifier, x, y):

    y_pred = classifier.predict(x)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # Calcula a curva ROC para cada classe, baseado na predição do modelo e a saída real
    for i in range(len(CLASSES)):
        fpr[i], tpr[i], _ = roc_curve((y==CLASSES[i])*1, (y_pred==CLASSES[i])*1)
        roc_auc[i] = auc(fpr[i], tpr[i])
    plt.figure(figsize=(16, 16))
    for i in range(len(CLASSES)):
        plt.plot(
            fpr[i],
            tpr[i],
            label="ROC curve of {0} (AUC = {1:0.2f})".format(CLASSES[i], roc_auc[i]))
    plt.plot([0, 1], [0, 1], "k--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")

    # Salva a imagem com o nome da avaliação e o com o sufixo "_roc_curve"
    image_path = os.path.join(RESULT_PATH,evaluation_name+'_roc_curve.png')
    plt.savefig(image_path)

    # Exibe a curva ROC para cada classe
    if VERBOSE['EVALUATION']:
        plt.show()
    else:
        plt.close()
        
    return fpr, tpr, roc_auc
def evaluate_model_by_class_feat_extract(classifier, x, y):
    def separate_by_class(x, y):

        n_classes = len(CLASSES)
        x_classified = [[] for _ in range(n_classes)]
        y_classified = [[] for _ in range(n_classes)]
        
        # Separa as classes
        for i,img in enumerate(y):
            index = np.where(np.array(CLASSES)==img)[0][0]
            x_classified[index].append(x[i])
            y_classified[index].append(img)

        # Converte a entrada e saída para um array numpy
        for i in range(n_classes):
            x_classified[i] = np.array(x_classified[i])
            y_classified[i] = np.array(y_classified[i])
            
        return np.array(x_classified,dtype=object), np.array(y_classified,dtype=object)

    x_by_class, y_by_class = separate_by_class(x,y)

    score_by_class = []

    # Avalia o modelo para cada classe
    for x_class, y_class in zip(x_by_class,y_by_class):
        predicted = classifier.predict(x_class)

        y_class.shape
        score = accuracy_score(predicted,y_class)
        score_by_class.append(score)

    return score_by_class

def evaluate_model_confusion_matrix_feat_extract(evaluation_name, classifier, x, y):
    y_pred = classifier.predict(x)

    # Cria a matriz de confusão baseado na previsão do modelo de classificação e o resultado real
    cm = confusion_matrix(y, y_pred)

    cm_df = pd.DataFrame(cm,
                        index = [CLASSES[i] for i in range(len(CLASSES))], 
                        columns = [CLASSES[i] for i in range(len(CLASSES))])
    plt.figure(figsize=(16,16))
    sns.heatmap(cm_df, annot=True)
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Values')
    plt.xlabel('Predicted Values')
    
    # Salva a imagem com o nome da avaliação e o com o sufixo "_confusion_matrix"
    image_path = os.path.join(RESULT_PATH,evaluation_name+'_confusion_matrix.png')
    plt.savefig(image_path)

    # Exibe a matriz de confusão
    if VERBOSE['EVALUATION']:
        plt.show()
    else:
        plt.close()
    return cm


def evaluate_feat_extract(evaluation_name, classifier, x, y):
    if x is None:
        return None

    predicted = classifier.predict(x)
    score = accuracy_score(predicted,np.ravel(y, order='C'))
    score_by_class = evaluate_model_by_class_feat_extract(classifier, x, y)
    cm = evaluate_model_confusion_matrix_feat_extract(evaluation_name, classifier,x, y)
    _, _, roc_auc = plot_evaluation_roc_feat_extract(evaluation_name, classifier, x, y)
    table = PrettyTable()
    table.add_column("Metrics", ["Accuracy"])
    table.add_column("Global", [np.round(score,4)])


    # Cria uma tabela dos resultados por classe
    for i, s_class in enumerate(score_by_class):
        table.add_column(CLASSES[i], [np.round(s_class,4)])
    if VERBOSE['EVALUATION']:
        print()
        print(evaluation_name)
        print(table)

    all_scores = {
        'name': evaluation_name,
        'loss': score,
        'loss_by_class': score_by_class,
        'confusion': cm.tolist(),
        'auc': list(roc_auc.values())
    }

    save_evaluation_feat_extract(evaluation_name, all_scores, table)

    return all_scores

In [8]:
def classify_feat_extract(classifier,x,y):
    classifier.fit(x,y)
    return classifier

def extract_and_evaluate2(model_and_parameter, classifier):
    x_train, y_train, x_test, y_test = feature_extraction(model_and_parameter)
    classifier_fit = classify_feat_extract(classifier(y_train),x_train,y_train)
    evaluate_feat_extract('train', classifier_fit, x_train, y_train)
    evaluate_feat_extract('test', classifier_fit, x_test, y_test)